In [1]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "cat_test")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'infer_merge'
# experiment_name = 'trigger_inference'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-d13'
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :", aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13",   
                                                                min_nodes=0, 
                                                                max_nodes=2,
                                               idle_seconds_before_scaledown=450)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
print("Azure Machine Learning Compute Cluster attached now")

# Or use an existing env which was registered to the ws
env_name = "vdsm_all" 
rebuild_env = False   # True if you want to rebuild this env
if rebuild_env:
    env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
    env.register(workspace=ws)
else:
    envs = Environment.list(workspace=ws)
    if env_name in envs:
        # Use an existing environment
        env = Environment.get(workspace=ws, name=env_name)
        print("using existing")
    else:
        # Create a new env using yml
        env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
        # Register the environment 
        env.register(workspace=ws)

This Compute Target already exist.
Azure Machine Learning Compute Cluster attached now
using existing


In [4]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="infer_merge.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=20,
                                        run_invocation_timeout=2400,
                                        node_count=2)

month_pipeline_param = PipelineParameter(name="month", default_value="2021-12")

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

merge_step = ParallelRunStep(
    name="merge_for_inference",
    inputs=ds_list,
    arguments=[
        "--month", month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

trigger_step = PythonScriptStep(
    name="trigger_fusion_pipeline",
    script_name="trigger_fusion_pipeline.py",
    compute_target=aml_compute,
    source_directory="."
    )

step_sequence = StepSequence(steps=[merge_step, trigger_step])  

pipeline = Pipeline(workspace=ws, steps=step_sequence)
# pipeline_run = exp.submit(pipeline)
# print('VDSM training data fusion submitted for execution')
# RunDetails(pipeline_run).show()
# pipeline_run.wait_for_completion()


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# pipeline.publish_pipeline(name="infer_merge")
pipeline.publish(name="infer_merge", description="Merge S2 data for inference", version=7)

Created step merge_for_inference [cb67ebc2][c55aaa7e-5bac-445e-97d4-983dea4be8d2], (This step will run and generate new outputs)Created step trigger_fusion_pipeline [431523e9][c6dcce6e-52ed-437d-b9e0-3df0c483f880], (This step will run and generate new outputs)



Pipeline(Name: infer_merge,
Id: 470f4f8d-2ea0-4578-9913-730259773874,
Status: Active,
Endpoint: https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/470f4f8d-2ea0-4578-9913-730259773874)

In [18]:
import requests
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
rest_endpoint='https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/ef55d26a-2d58-4bd9-b21f-e44d89c35bb4'
response = requests.post(rest_endpoint, headers=auth_header, json={"ExperimentName": "infer_merge", "ParameterAssignments": {"process_count_per_node": 6, "month": "2021-07"}})

In [11]:
response

<Response [200]>

In [6]:
import requests
from azureml.core.authentication import ServicePrincipalAuthentication

# interactive_auth = InteractiveLoginAuthentication()
svc_pr = ServicePrincipalAuthentication(
       tenant_id="4341df80-fbe6-41bf-89b0-e6e2379c9c23",
       service_principal_id="0f160390-2fa0-4918-a2b7-c7a0c05464f0",
       service_principal_password="ffb5ad7d-5ced-4364-8375-393fc6d2f2ca")
# auth_header = interactive_auth.get_authentication_header()
auth_header = svc_pr.get_authentication_header()
rest_endpoint='https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/b4ef7431-e393-406a-a524-220ec9cdb205'
# response = requests.post(rest_endpoint, headers=auth_header, json={"ExperimentName": "infer_fusion", "ParameterAssignments": {"process_count_per_node": 6, "month": "2021-08", "random_n": -1, "pv": True, "npv": False}})
response = requests.post(rest_endpoint, json={"ExperimentName": "infer_fusion", "ParameterAssignments": {"process_count_per_node": 6, "month": "2021-08", "random_n": -1, "pv": True, "npv": False}})

AuthenticationError: Get Token request returned http error: 401 and server response: {"error":"invalid_client","error_description":"AADSTS7000215: Invalid client secret is provided.\r\nTrace ID: ab952e1b-52bb-4b96-b417-135fd5952600\r\nCorrelation ID: 4c08be2a-6d5d-43cb-bcb3-737c1a63771d\r\nTimestamp: 2022-01-13 00:30:35Z","error_codes":[7000215],"timestamp":"2022-01-13 00:30:35Z","trace_id":"ab952e1b-52bb-4b96-b417-135fd5952600","correlation_id":"4c08be2a-6d5d-43cb-bcb3-737c1a63771d","error_uri":"https://login.microsoftonline.com/error?code=7000215"}

In [5]:
response

<Response [401]>